In [1]:
import tensorflow as tf
print("Using TensorFlow version %s" % tf.__version__)

Using TensorFlow version 2.5.0


In [2]:
import numpy as np

##12.2 넘파이처럼 텐서플로 사용하기

###12.2.1 텐서와 연산

In [3]:
tf.constant([[1., 2., 3.],
             [4., 5., 6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
# tf.convert_to_tensor
_array_with_reshape = np.arange(1, 7, dtype=np.float32).reshape((2, -1))
tf.convert_to_tensor(_array_with_reshape)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [5]:
_array = np.arange(1, 7, dtype=np.float32)
# tf.convert_to_tensor
_tf_array = tf.convert_to_tensor(_array)
# tf.reshape
_tf_reshaped = tf.reshape(_tf_array, (2, 3))
_tf_reshaped

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [6]:
tf.experimental.numpy?

In [7]:
# dtype, shape
_tf_reshaped.dtype, _tf_reshaped.shape

(tf.float32, TensorShape([2, 3]))

In [8]:
# indexing
_tf_reshaped[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [9]:
# tf.newaxis
_tf_reshaped[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

###12.2.2 텐서와 넘파이

In [10]:
# numpy array to tensor
a = np.array([2, 4, 5], dtype=np.float64)
t = tf.constant(a)
t

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [11]:
# tensor to numpy array
t.numpy()

array([2., 4., 5.])

###12.2.3 타입 변환

In [12]:
t.dtype

tf.float64

In [13]:
# tf.cast?

In [14]:
t = tf.cast(t, tf.float32) # inplace=False
t.dtype

tf.float32

###12.2.4 변수

In [15]:
_arr = np.arange(1, 7, dtype=np.float32).reshape((2, -1))
v = tf.Variable(_arr, name='test_variable')
v

<tf.Variable 'test_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [16]:
v * 2
v

<tf.Variable 'test_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [17]:
# assign ; inplace=True
v.assign(v * 2)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [18]:
# indexing하여 assign
# broadcasting❌
v[:, 2].assign([77, 77])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4., 77.],
       [ 8., 10., 77.]], dtype=float32)>

###12.2.5 다른 데이터 구조


* 희소 텐서
* 텐서 배열
* 래그드 텐서
* 문자열 텐서
* 집합
* 큐

##12.3 사용자 정의 모델과 훈련 알고리즘

###12.3.1 사용자 정의 손실 함수

In [19]:
tf.keras.losses.Huber?

In [20]:
# tf.where?

In [21]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - .5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [22]:
# huber function standalone usage
np.random.seed(777)
y_true = np.random.randint(0, 5, 10)
y_true = y_true.astype(np.float64)
error = np.random.rand(10) * 2
y_pred = y_true + error
huber_fn(tf.constant(y_true), tf.constant(y_pred))

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([0.01269968, 0.67922751, 0.23576545, 1.4777523 , 0.75294641,
       0.86355856, 0.60451363, 0.14457146, 0.27864514, 0.09939387])>

In [23]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - threshold**2*0.5
        return tf.where(is_small_error, squared_loss, linear_loss)

In [24]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - self.threshold**2*0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}

In [25]:
hloss = HuberLoss(name='hloss')

In [26]:
hloss.get_config()

{'name': 'hloss', 'reduction': 'auto', 'threshold': 1.0}

In [27]:
def func(a, b, c):
    return a + b * c

In [28]:
abc_params = {'a':1, 'b':2, 'c':3}

In [29]:
func(**abc_params)

7

###12.3.3 활성화 함수, 초기화, 규제, 제한을 커스터마이징하기

In [30]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [31]:
_weights = np.random.randint(1,10, 12).reshape([2, 2, 3])
_weights

array([[[8, 9, 1],
        [9, 4, 3]],

       [[1, 4, 4],
        [5, 1, 7]]])

In [32]:
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"facotr":self.factor}

In [33]:
my_l1_regularizer = MyL1Regularizer(2.)
my_l1_regularizer.__call__(_weights)

<tf.Tensor: shape=(), dtype=float64, numpy=112.0>

In [34]:
my_l1_regularizer.get_config()

{'facotr': 2.0}

###12.3.4 사용자 정의 지표

In [35]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1., **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}

In [36]:
tf.keras.metrics.Metric.add_weight?

###12.3.5 사용자 정의 층

In [37]:
# 가중치가 없는 레이어
exp_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))
exp_layer(tf.constant(0.))

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [38]:
tf.keras.activations.get('sigmoid')

<function tensorflow.python.keras.activations.sigmoid>

In [39]:
# 가중치를 갖는 레이어: MyDense
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)
    
    def call(self, X):
        return self.activation(X @ self.kernerl + self.bias) 
                                # @; matmul

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
                              # input_shape                       # output_shape
    
    def get_config(self):
        base_config = super().__init__(**kwargs)
        return {**base_config, 
                "units":self.units,
                "activation":tf.keras.activations.serialize(self.activation)}
                                                  # 활성함수 전체설정 저장

In [40]:
tf.keras.layers.Layer.add_weight?

In [41]:
tf.constant(np.arange(1, 7).reshape((3, 2))).shape.as_list()[:-1]+[10]
                                            # input_shape        # output_shape

[3, 10]

In [42]:
tf.keras.layers.GaussianNoise

tensorflow.python.keras.layers.noise.GaussianNoise

###12.3.6 사용자 정의 모델

In [43]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation='elu',
                                           kernel_initializer='he_normal')
        for _ in range(n_layers)]
    
    def call(self, input):
        Z = inputs # resiudal
        for layer in self.hidden:
            Z = layer(Z)
        Z += inputs
        return Z

In [44]:
class ResidualRegressor(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation='elu', 
                                             kernel_initializer='he_normal')
        self.block1 = ResidualBlcok(2, 30)
        self.block2 = ResidualBlcok(2, 30)
        self.out = tf.keras.layers.Dense(output_dim)
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1+3):
            Z = block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

###12.3.7 모델 구성 요소에 기반한 손실과 지표

In [45]:
class ReconstructionRegressor(tf.keras.Model):
    def __init__(self, output_dims, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = [tf.keras.layers.Dense(30, activation='selu', 
                                              kernel_initializer='lecun_normal') 
        for _ in range(5)]
        self.out = tf.keras.layers.Dense(output_dims)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruction = tf.keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden1:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(inputs - reconstruction))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)

###12.3.8 자동 미분을 사용하여 그레이디언트 계산하기

In [46]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

In [47]:
w1, w2 = 5, 3

In [48]:
eps = 1e-6

In [49]:
(f(w1+eps, w2) - f(w1, w2)) / eps

36.000003007075065

In [50]:
(f(w1, w2+eps) - f(w1, w2)) / eps

10.000000003174137

In [51]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)

In [52]:
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [53]:
def f(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

In [54]:
with tf.GradientTape() as tape:
    z = f(w1, w2)

In [55]:
gradients = tape.gradient(z, [w1, w2])

###12.3.9 사용자 정의 훈련 반복

In [56]:
l2_reg = tf.keras.regularizers.l2(0.05)
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(30, activation='elu', 
                                                          kernel_initializer='he_normal',
                                                          kernel_regularizer=l2_reg),
                                    tf.keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

In [57]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [58]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = ' - '.join([f"{m.name}: {m.result():.4f}"
    for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print(f"\r{iteration}/{total} " + metrics, end=end)

In [59]:
(X_train, y_train), (X_valid, y_valid) = tf.keras.datasets.boston_housing.load_data()

In [60]:
n_epochs = 20
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = tf.keras.losses.mean_squared_error
mean_loss = tf.keras.metrics.Mean()
metrics = [tf.keras.metrics.MeanAbsoluteError()]

In [61]:
for epoch in range(1, n_epochs+1):
    print(f"Epoch {epoch}/{n_epochs}")
    for step in range(1, n_steps+1):
        X_batch, y_batch = random_batch(X_train, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_state()

Epoch 1/20
404/404 mean: 7445.8696 - mean_absolute_error: 61.4298
Epoch 2/20
404/404 mean: 288.5289 - mean_absolute_error: 10.4735
Epoch 3/20
404/404 mean: 146.8493 - mean_absolute_error: 8.4159
Epoch 4/20
404/404 mean: 130.0673 - mean_absolute_error: 7.4215
Epoch 5/20
404/404 mean: 123.7450 - mean_absolute_error: 6.8052
Epoch 6/20
404/404 mean: 117.2950 - mean_absolute_error: 7.2581
Epoch 7/20
404/404 mean: 111.3075 - mean_absolute_error: 6.7560
Epoch 8/20
404/404 mean: 125.5476 - mean_absolute_error: 7.4403
Epoch 9/20
404/404 mean: 121.5084 - mean_absolute_error: 7.0688
Epoch 10/20
404/404 mean: 116.2579 - mean_absolute_error: 7.1123
Epoch 11/20
404/404 mean: 119.2943 - mean_absolute_error: 7.4660
Epoch 12/20
404/404 mean: 103.7029 - mean_absolute_error: 6.4544
Epoch 13/20
404/404 mean: 97.9774 - mean_absolute_error: 6.5304
Epoch 14/20
404/404 mean: 116.9741 - mean_absolute_error: 7.1519
Epoch 15/20
404/404 mean: 121.5855 - mean_absolute_error: 7.3557
Epoch 16/20
404/404 mean: 135.54